# A simple RAG application using open-source models

In [1]:
import os
MODEL = "llama2"


In [2]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [4]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t\nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [5]:
chain = prompt | model | parser

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("hackmet.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='{ \n    "page_1" : "Materials Science and Engineering A 393 (2005) 42 \\u201350\\nThe \nrole of heat treatment on the erosion \\u2013corrosion behavior \\nof AISI 52100 \nsteel\\nJianhui Xie \\u2217, Ahmet T. Alpas, Derek O. Northwood \\nDepartment of \nMechanical, Automotive and Materials Engineering, University of Windsor, 401 \nSunset Avenue, \\nWindsor, ON, Canada N9B 3P4 \\nReceived 18 September 2003; \nreceived in revised form 23 September 2004; accepted 24 September \n2004\\nAbstract \\nTheroleofheattreatmentontheerosionandcorrosionbehaviorof52100s\nteelwasinvestigatedintermsof:(i)electrochemicalparameter \\nmeasurements,(ii)ide\nnti\\ufb01cationofmicrostrainandstrainenergyinducedbyerosion,and(iii)thecharact\nerizationofsynergisticeffectbetwee n \\nerosion and corrosion. Heat-treatment \nboth thermodynamically and kinetically affects the electrochemical reaction of \neroded 52100 \nsteel.\\nBasedonboththepolarizationresistanceandcorrosionratemeasurements,

In [16]:
pages[0].page_content

'{ \n    "page_1" : "Materials Science and Engineering A 393 (2005) 42 \\u201350\\nThe \nrole of heat treatment on the erosion \\u2013corrosion behavior \\nof AISI 52100 \nsteel\\nJianhui Xie \\u2217, Ahmet T. Alpas, Derek O. Northwood \\nDepartment of \nMechanical, Automotive and Materials Engineering, University of Windsor, 401 \nSunset Avenue, \\nWindsor, ON, Canada N9B 3P4 \\nReceived 18 September 2003; \nreceived in revised form 23 September 2004; accepted 24 September \n2004\\nAbstract \\nTheroleofheattreatmentontheerosionandcorrosionbehaviorof52100s\nteelwasinvestigatedintermsof:(i)electrochemicalparameter \\nmeasurements,(ii)ide\nnti\\ufb01cationofmicrostrainandstrainenergyinducedbyerosion,and(iii)thecharact\nerizationofsynergisticeffectbetwee n \\nerosion and corrosion. Heat-treatment \nboth thermodynamically and kinetically affects the electrochemical reaction of \neroded 52100 \nsteel.\\nBasedonboththepolarizationresistanceandcorrosionratemeasurements,itwas\nfoundthatatemper

In [17]:
type(pages)

list

In [18]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents([pages[0]], embedding=embeddings)

In [ ]:
retriever = vectorstore.as_retriever()
# retriever.invoke("machine learning")

In [ ]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [ ]:
questions = [
    "Predict heat treatment schedule for tempering steel",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()